# Step 3-4. 텍스트로 이미지 검색 (크로스모달 검색)
텍스트 키워드로 유사한 이미지를 검색합니다. **Titan과 Nova 두 모델의 검색 결과를 비교**합니다.

## Titan vs Nova 텍스트 검색 비교

| | Titan (Step 3-2a) | Nova (Step 3-2b) |
|---|---|---|
| **모델 ID** | `amazon.titan-embed-image-v1` | `amazon.nova-2-multimodal-embeddings-v1:0` |
| **인덱스** | `titan-image-test` | `nova-image-test` |
| **텍스트 언어** | 영어만 | **한국어 포함 200개 언어** |

## 실습 과제
- 영어 쿼리(예: "cute dog")와 한국어 쿼리(예: "귀여운 강아지")로 각각 검색하여 두 모델의 결과를 비교해보세요
- Titan은 영어에서만 정확한 결과를 반환하고, 한국어 쿼리에서는 유사도 점수가 비슷하게 나와 구분이 어렵습니다
- Nova는 영어/한국어 모두 정확한 결과를 반환합니다

In [ ]:
!pip install -q boto3==1.38.46 opensearch-py==2.8.0

## 1. 설정 (Configuration)

In [27]:
import os, json

# Step 0에서 저장한 설정 불러오기
try:
    with open("../config.json") as f:
        _config = json.load(f)
    print("✅ config.json 로드 완료")
except FileNotFoundError:
    raise FileNotFoundError("❌ config.json을 찾을 수 없습니다. Step 0 노트북을 먼저 실행해주세요.")

HOST = _config.get("OPENSEARCH_HOST")
if not HOST:
    raise ValueError("❌ config.json에 OPENSEARCH_HOST 값이 없습니다. Step 0 노트북을 먼저 실행해주세요.")
DEFAULT_REGION = _config.get("DEFAULT_REGION", "ap-northeast-2")
BEDROCK_REGION = _config.get("BEDROCK_REGION", "us-east-1")
PROFILE = _config.get("PROFILE", "skku-opensearch-session")

# Titan (영어 전용)
TITAN_INDEX_NAME = 'titan-image-test'
TITAN_MODEL_ID = 'amazon.titan-embed-image-v1'

# Nova (한국어 포함 다국어 지원)
NOVA_INDEX_NAME = 'nova-image-test'
NOVA_MODEL_ID = 'amazon.nova-2-multimodal-embeddings-v1:0'

VECTOR_DIMENSION = 1024

✅ config.json 로드 완료


## 2. OpenSearch 클라이언트 생성

In [28]:
import boto3
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection

service = 'aoss'
credentials = boto3.Session(profile_name=PROFILE).get_credentials()
auth = AWSV4SignerAuth(credentials, DEFAULT_REGION, service)

client = OpenSearch(
    hosts=[{'host': HOST, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

print("OpenSearch 클라이언트 생성 완료")

OpenSearch 클라이언트 생성 완료


## 3. Bedrock 클라이언트 생성 및 텍스트 임베딩 함수 정의
Titan과 Nova는 API 형식이 다르므로 각각의 임베딩 함수를 정의합니다.

| | Titan | Nova |
|---|---|---|
| **요청 형식** | `{"inputText": "..."}` | `{"taskType": "SINGLE_EMBEDDING", ...}` |
| **응답 형식** | `response["embedding"]` | `response["embeddings"][0]["embedding"]` |

In [29]:
import json
import math

session = boto3.Session(profile_name=PROFILE)
bedrock_client = session.client(
    service_name='bedrock-runtime',
    region_name=BEDROCK_REGION,
)
print("Bedrock client created successfully.")

def normalize_vector(vec):
    """벡터를 L2 정규화합니다."""
    norm = math.sqrt(sum(x * x for x in vec))
    return [x / norm for x in vec] if norm > 0 else vec

def get_titan_text_embedding(text):
    """Titan 멀티모달 모델로 텍스트 임베딩을 반환합니다. (영어 전용)"""
    body = json.dumps({"inputText": text})
    response = bedrock_client.invoke_model(
        body=body,
        modelId=TITAN_MODEL_ID,
        accept="application/json",
        contentType="application/json",
    )
    response_body = json.loads(response.get("body").read())
    return normalize_vector(response_body.get("embedding"))

def get_nova_text_embedding(text):
    """Nova 멀티모달 모델로 텍스트 임베딩을 반환합니다. (한국어 포함 다국어)"""
    body = json.dumps({
        "taskType": "SINGLE_EMBEDDING",
        "singleEmbeddingParams": {
            "embeddingPurpose": "GENERIC_RETRIEVAL",
            "embeddingDimension": VECTOR_DIMENSION,
            "text": {
                "truncationMode": "END",
                "value": text
            }
        }
    })
    response = bedrock_client.invoke_model(
        body=body,
        modelId=NOVA_MODEL_ID,
        accept="application/json",
        contentType="application/json",
    )
    response_body = json.loads(response.get("body").read())
    return normalize_vector(response_body["embeddings"][0]["embedding"])

Bedrock client created successfully.


In [30]:
# 두 인덱스 모두 데이터가 업로드되었는지 확인
for name, step in [(TITAN_INDEX_NAME, "3-2a (Titan)"), (NOVA_INDEX_NAME, "3-2b (Nova)")]:
    if not client.indices.exists(index=name):
        print(f"❌ '{name}' 인덱스가 존재하지 않습니다.")
        print(f"   → Step {step} 노트북을 먼저 실행해주세요.")
    else:
        doc_count = client.count(index=name)['count']
        if doc_count == 0:
            print(f"⏳ '{name}' 인덱스는 있지만 검색 가능한 문서가 0개입니다.")
            print("   → 데이터 업로드 직후라면 인덱싱 중일 수 있습니다. 잠시 후 다시 실행해주세요.")
        else:
            print(f"✅ '{name}' 인덱스에 {doc_count}개의 문서가 준비되어 있습니다.")

❌ 'titan-image-test' 인덱스가 존재하지 않습니다.
   → Step 3-2a (Titan) 노트북을 먼저 실행해주세요.
✅ 'nova-image-test' 인덱스에 12개의 문서가 준비되어 있습니다.


## 4. 텍스트 벡터 변환 및 이미지 검색 (Titan vs Nova 비교)
같은 쿼리 텍스트로 Titan 인덱스와 Nova 인덱스를 각각 검색하여 결과를 비교합니다.

**실습 포인트:**
- 영어 쿼리(예: `"cute dog"`)로 검색하면 두 모델 모두 비슷한 결과를 보입니다
- 한국어 쿼리(예: `"귀여운 강아지"`)로 검색하면 Titan은 유사도 점수가 모두 비슷하게 나와 구분이 안 되지만, Nova는 정확한 결과를 반환합니다

In [31]:
from opensearchpy.exceptions import RequestError

# TODO: 검색할 텍스트를 바꿔보세요!
QUERY_TEXT = "귀여운 강아지"

K_NEIGHBORS = 5
RESULT_SIZE = 5

search_models = [
    {"name": "Titan (영어 전용)", "index": TITAN_INDEX_NAME, "embed_fn": get_titan_text_embedding},
    {"name": "Nova (다국어 지원)", "index": NOVA_INDEX_NAME, "embed_fn": get_nova_text_embedding},
]

for model in search_models:
    print(f"\n{'='*60}")
    print(f"🔍 [{model['name']}] '{QUERY_TEXT}' 검색")
    print(f"   인덱스: {model['index']}")
    print(f"{'='*60}")

    if not client.indices.exists(index=model['index']):
        print(f"⚠️ '{model['index']}' 인덱스가 없습니다. 스킵합니다.")
        continue

    try:
        query_vector = model['embed_fn'](QUERY_TEXT)

        search_query = {
            "size": RESULT_SIZE,
            "query": {
                "knn": {
                    "content_vector": {
                        "vector": query_vector,
                        "k": K_NEIGHBORS
                    }
                }
            }
        }

        response = client.search(index=model['index'], body=search_query)
        hits = response['hits']['hits']

        if not hits:
            print("검색된 이미지가 없습니다.")
        else:
            for i, hit in enumerate(hits):
                score = hit['_score']
                image_path = hit['_source'].get('image_path', 'N/A')
                print(f"  [{i + 1}] 유사도: {score:.4f}  |  {image_path}")

    except RequestError as e:
        print(f"  [!!!] 검색 중 에러가 발생했습니다: {e.info}")
    except Exception as e:
        print(f"  [!!!] 예상치 못한 에러가 발생했습니다: {e}")


🔍 [Titan (영어 전용)] '귀여운 강아지' 검색
   인덱스: titan-image-test
⚠️ 'titan-image-test' 인덱스가 없습니다. 스킵합니다.

🔍 [Nova (다국어 지원)] '귀여운 강아지' 검색
   인덱스: nova-image-test
  [1] 유사도: 1.4237  |  ../../data/image/dog_2.jpg
  [2] 유사도: 1.4234  |  ../../data/image/dog_3.jpg
  [3] 유사도: 1.2646  |  ../../data/image/cat_3.jpg
  [4] 유사도: 1.2642  |  ../../data/image/cat_1.jpg
  [5] 유사도: 1.2535  |  ../../data/image/dog_1.png
